# Time-Weighted Average Price (TWAP) v1 using minute bar data

## Parameters
Parameters: &lt;side&gt; &lt;quantity&gt; [limit_px] 

side = "b" (buy) or "s" (sell)

quantity = # of shares

limit_px = limit price (optional)

### Structure
1. Get market data
1. Read order parameters 
1. Calculate shares per bin to execute
1. Initiate loop (reading market data)
    1. Read each n bins of data
    1. Record a trade at the bin price (last for now)
    1. Accumulate the trade per bin
1. Calculate and plot summary statistics (average price, slippage vs. vwap, vs. arrival, vs. interval vwap

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%run getstock.py

In [ ]:
# get our market data, plot it to sanity check
bars = getMinuteStockPrices(symbol='AAPL')
bars.head()

In [ ]:
day = bars['2019-04-11']

In [ ]:
day['close'].plot(figsize=(16,9))

## Time-Weighted Average Price (TWAP) v1.0

In [ ]:
# order parameters
quantity = 1000000
quantity_remaining = quantity
trades = pd.DataFrame( columns = ['price' , 'shares'], index=day.index )

# v1.0 
# assume this is a full day order
# use 1 minute bins
n_bins = 390
slice_size = quantity / n_bins

# main event loop
for index, row in day.iterrows() :
    # Read each bin of data
    px = (row['high'] + row['low']) / 2 # calculate midpoint of that bar
    volume = row['volume']
    
    # Calculate % of volume and bin price (last for now)
    new_trade = slice_size
    
    # Accumulate the trade per bin
    trades.loc[index] = [px, new_trade]
    
    # decrement the remaining qty, and bail out if it's zero
    quantity_remaining -= new_trade
    
    # debug
    #print( str(volume) + " " + str(new_trade) + " " + str(quantity_remaining) )
    
    if (quantity_remaining <= 0):
        break

In [ ]:
trades['shares'].sum()

### Performance Analysis of TWAP

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
day['close'].plot(ax=ax1, color='k')
trades['price'].plot(ax=ax1, color='r')
plt.show()

### Summary Statistics of TWAP Order

In [ ]:
# average trade price
pavg = (trades['price'] * trades['shares'] ).sum() /  trades['shares'].sum()

# calculate VWAP
vwap = (day['close'] * day['volume']).sum() / day['volume'].sum()

slippage =  ((vwap - pavg) / vwap) * 10000

print("VWAP: %.4f" % vwap)
print("Pavg: %.4f" % pavg)
print("Performance vs Day VWAP in bps: %.4f" % slippage)